In [ ]:
# ============================================
# ライブラリのインポートと初期設定
# ============================================

# 時間管理とシステム操作用
import time 
from subprocess import call
import os
import sys
import copy

# 数値計算・データ処理用
import numpy as np  # 数値計算の基本ライブラリ
import matplotlib.pyplot as plt  # グラフ描画用
import h5py  # HDF5形式のデータファイル読み書き用

# 進捗表示と画像処理
from tqdm import tqdm  # プログレスバーの表示
from PIL import Image  # 画像処理用

# 機械学習関連
from sklearn.preprocessing import StandardScaler  # データの標準化（平均0、標準偏差1に変換）

# データセット作成用
import webdataset as wds  # 大規模データセットをtar形式で効率的に作成

# Natural Scenes Dataset (NSD) へのアクセス
# nsd_accessはこちらから入手: https://github.com/tknapen/nsd_access
# データのダウンロード方法: https://cvnlab.slite.page/p/dC~rBTjqjb/How-to-get-the-data
from nsd_access import NSDAccess
nsd_path = '/scratch/gpfs/KNORMAN/natural-scenes-dataset'  # NSDデータの保存先パス
nsda = NSDAccess(nsd_path)  # NSDデータアクセス用オブジェクトの初期化

# 脳画像データ（NIfTI形式）の読み込み用
import nibabel as nib

# PyTorch: 深層学習フレームワーク
import torch
# GPUが利用可能ならGPU、なければCPUを使用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:",device)

In [ ]:
# ============================================
# データ保存先とShared1000データの読み込み
# ============================================

# 作業用の一時ディレクトリパス
# 処理したデータセットの保存先として使用
tmp = '/scratch/gpfs/KNORMAN'

# Shared1000データの読み込み
# shared1000: 全被験者が共通で見た1000枚の画像のインデックス情報
# このデータは評価（テスト）用セットとして使用される
# True/Falseの配列で、各COCO画像がshared1000に含まれるかを示す
# ダウンロード先: https://huggingface.co/datasets/pscotti/mindeyev2/tree/main
shared1000 = np.load("shared1000.npy")

In [ ]:
# ============================================
# メインの処理ループ: fMRIデータの前処理とデータセット作成
# ============================================

# 被験者のループ（現在は被験者01のみ処理、コメントアウトを外すと全8名処理可能）
for sub in [0]: #,1,2,3,4,5,6,7]:
    # 被験者IDの設定（subj01, subj02, ...）
    subject=f'subj0{sub+1}'
    subj=subject
    print(subject)
    
    # カウンター変数の初期化
    abs_cnt = -1  # 全トライアルの絶対カウンター
    abs_notshared1000_cnt = -1  # Shared1000以外のトライアルカウンター（訓練用）
    abs_shared1000_cnt = -1  # Shared1000トライアルカウンター（テスト用）
    
    # ============================================
    # 1. COCO 73k画像インデックスの読み込み
    # ============================================
    # 各被験者がどのCOCO画像を見たかの情報を読み込む
    indices_path = "COCO_73k_subj_indices.hdf5"
    hdf5_file = h5py.File(indices_path, "r")
    indices = hdf5_file[f"{subj}"][:]

    # ============================================
    # 2. セッション数とトライアル数の設定
    # ============================================
    # 各被験者のセッション数（fMRIスキャンの回数）
    # 被験者によってセッション数が異なる（30〜40回）
    nsessions_allsubj=np.array([40, 40, 32, 30, 40, 32, 40, 30]) 
    nsessions=nsessions_allsubj[sub]  # 現在の被験者のセッション数
    ntrials = 750*nsessions  # 総トライアル数（1セッションあたり750トライアル）
    print(nsessions,ntrials)

    print(time.strftime("\nCurrent time: %H:%M:%S", time.localtime())) 
    
    # ============================================
    # 3. 脳のROI（関心領域）マスクの読み込み
    # ============================================
    # nsdgeneral: 脳全体をカバーする広い領域のマスク
    file = f"/scratch/gpfs/KNORMAN/natural-scenes-dataset/nsddata/ppdata/{subject}/func1pt8mm/roi/nsdgeneral.nii.gz"
    nifti = nib.load(file) 
    mask = nifti.get_data()
    mask[mask<1] = 0  # 0以下の値を0に設定
    nsdgeneral_mask = mask

    # ============================================
    # 4. 各セッションのfMRIデータと行動データの読み込み・前処理
    # ============================================
    for tar in tqdm(range(nsessions)):
        sess=tar+1  # セッション番号（1から開始）
        
        # 行動データの読み込み（被験者の応答、反応時間など）
        behav = nsda.read_behavior(subject=subject, 
                    session_index=sess, 
                    trial_index=[])  # 空リスト = 全トライアル取得

        # ============================================
        # fMRI Beta値の読み込みと前処理
        # ============================================
        # Beta値: GLMSingle手法で推定された、各ボクセルの画像に対する応答強度
        betas = nsda.read_betas(subject=subject, 
                            session_index=sess, 
                            trial_index=[],  # 空リスト = このセッションの全トライアル取得
                            data_type='betas_fithrf',  # GLMSingle beta2（HRF最適化済み）
                            data_format='func1pt8mm')  # 1.8mm解像度

        # 軸の入れ替え: 最後の次元（トライアル）を最初に移動
        betas = np.moveaxis(betas,-1,0)
        
        # ============================================
        # 高品質ボクセルの選択（ncSNRによるフィルタリング）
        # ============================================
        # ncSNR (noise ceiling SNR): ボクセルの信号品質を示す指標
        vox_include = copy.deepcopy(nsdgeneral_mask)
        ncsnr = nib.load(f"{subject}_ncsnr.nii.gz").get_fdata()
        ncsnr[ncsnr<.15] = np.nan  # ncSNR < 0.15のボクセルをNaNに設定
        if tar==0: print("voxels left:", len(vox_include[vox_include>0]))
        
        # 低品質ボクセルを除外（ただしnsdgeneralマスク内のボクセルは維持）
        vox_include[np.isnan(ncsnr)] -= 1
        vox_include[vox_include<0] = 0
        if tar==0: print("voxels left after ncsnr thresholding:", len(vox_include[vox_include>0]))  # subj01 = 約49329個
        
        # ============================================
        # データの標準化（Z-score正規化）
        # ============================================
        betas = betas.reshape(len(betas),-1)  # 2次元に変形
        betas = betas[:,vox_include.flatten().astype(bool)]  # 選択されたボクセルのみ抽出
        shape = betas.shape
        
        # 注意: 本来はShared1000データを除外して標準化すべき（MindEye2論文では実施）
        scalar = StandardScaler(with_mean=True, with_std=True).fit(betas)
        betas_mean = scalar.mean_
        betas_std = scalar.scale_
        betas = (betas - betas_mean) / betas_std  # 平均0、標準偏差1に正規化
        betas = betas.reshape(shape).astype('float16')  # メモリ節約のためfloat16に変換
        
        # セッションごとにグローバル変数として保存
        globals()[f'betas_ses{sess}'] = betas  
        globals()[f'behav_ses{sess}'] = behav   
        print(betas.shape)
        
    # ============================================
    # 5. 全セッションのBeta値を1つのファイルに結合
    # ============================================
    for tar in range(nsessions):
        sess=tar+1
        
        if sess==1:
            betas_all = globals()[f'betas_ses{sess}']
        else:
            betas_all = np.vstack((betas_all,globals()[f'betas_ses{sess}']))  # 縦方向に結合
        print(betas_all.shape)
        
    # HDF5形式で保存（大容量データの効率的な保存形式）
    with h5py.File(f'betas_{subject}.hdf5', 'w') as f:
        f.create_dataset('betas', data=betas_all)
    print(f"saved betas_{subject}.hdf5")
        
    # ============================================
    # 6. WebDataset形式でのデータセット作成（行動データ）
    # ============================================
    # ディレクトリ構造の作成
    os.makedirs(f"{tmp}/mindeyev2_wds/{subj}",exist_ok=True)
    os.makedirs(f"{tmp}/mindeyev2_wds/{subj}/train",exist_ok=True)  # 訓練用
    os.makedirs(f"{tmp}/mindeyev2_wds/{subj}/test",exist_ok=True)  # テスト用（Shared1000）
    
    # テスト用のTarファイルライター
    sink1 = wds.TarWriter(f"{tmp}/mindeyev2_wds/{subj}/test/0.tar")
    
    # セッションごとに処理
    for tar in tqdm(range(nsessions)):
        behav = globals()[f'behav_ses{tar+1}']
        
        # 訓練用のTarファイルライター（セッションごとに1ファイル）
        sink2 = wds.TarWriter(f"{tmp}/mindeyev2_wds/{subj}/train/{tar}.tar")
        
        # 各トライアルを処理
        for i in range(len(behav)):
            abs_cnt += 1  # 絶対トライアル番号をインクリメント

            # ============================================
            # 同じ画像を見た過去のトライアルを検索
            # ============================================
            # NSDでは被験者は同じ画像を複数回見る（記憶実験のため）
            trial_numbers = np.where(indices==indices[abs_cnt])[0]
            assert np.isin(abs_cnt,trial_numbers)
            trial_numbers[trial_numbers == abs_cnt] = -1  # 現在のトライアルを-1にマーク
            # 最大3個のトライアル番号を保持（不足分は-1で埋める）
            if len(trial_numbers) == 1:
                trial_numbers = np.append(trial_numbers, -1)
                trial_numbers = np.append(trial_numbers, -1)
            if len(trial_numbers) == 2:
                trial_numbers = np.append(trial_numbers, -1)
            assert len(trial_numbers) == 3

            sess=tar+1
            behav = globals()[f'behav_ses{sess}']
            
            # ============================================
            # 7. 現在のトライアルの行動データを収集
            # ============================================
            # 行動データは17次元のベクトルとして保存
            behav_matrix = np.ones((1, 17))*-1  # -1で初期化（欠損値を示す）
            jjj=-1
            for j in range(1):
                jj = i-j
                jjj += 1

                if jj >= 0:
                    # NaN（欠損値）を-1に変換して保存
                    iscorrect = behav.iloc[jj]['ISCORRECT']  # 記憶課題の正誤
                    if np.isnan(iscorrect): iscorrect = -1

                    isoldcurrent = behav.iloc[jj]['ISOLDCURRENT']
                    if np.isnan(isoldcurrent): isoldcurrent = -1

                    iscorrectcurrent = behav.iloc[jj]['ISCORRECTCURRENT']
                    if np.isnan(iscorrectcurrent): iscorrectcurrent = -1

                    rt = behav.iloc[jj]['RT']  # 反応時間
                    if np.isnan(rt): rt = -1

                    changemind = behav.iloc[jj]['CHANGEMIND']  # 回答変更の有無
                    if np.isnan(changemind): changemind = -1

                    button = behav.iloc[jj]['BUTTON']  # 押されたボタン
                    if np.isnan(button): button = -1

                    total1 = behav.iloc[jj]['TOTAL1']
                    if np.isnan(total1): total1 = -1

                    total2 = behav.iloc[jj]['TOTAL2']
                    if np.isnan(total2): total2 = -1
                    
                    coco73 = int(behav.iloc[jj]['73KID'])-1  # COCO画像のインデックス（0始まり）
                    assert coco73 >= 0 and coco73 < 730000

                    # 行動データを辞書形式で整理
                    behavior = {
                        "cocoidx": coco73,                         #0  COCO画像インデックス
                        "subject": sub+1,                          #1  被験者番号
                        "session": int(behav.iloc[jj]['SESSION']), #2  セッション番号
                        "run": int(behav.iloc[jj]['RUN']),         #3  ラン番号
                        "trial": int(behav.iloc[jj]['TRIAL']),     #4  トライアル番号
                        "global_trial": (int(behav.iloc[jj]['SESSION'])-1)*750 + jj,  #5  全体でのトライアル番号
                        "time": int(behav.iloc[jj]['TIME']),       #6  時刻情報
                        "isold": int(behav.iloc[jj]['ISOLD']),     #7  既出画像かどうか
                        "iscorrect": iscorrect,                    #8  正答/誤答
                        "rt": rt,                                  #9  反応時間（0=反応なし）
                        "changemind": changemind,                  #10 回答変更
                        "isoldcurrent": isoldcurrent,              #11
                        "iscorrectcurrent": iscorrectcurrent,      #12
                        "total1": total1,                          #13
                        "total2": total2,                          #14
                        "button": button,                          #15 押されたボタン
                        "shared1000": shared1000[int(behav.iloc[jj]['73KID'])-1], #16 Shared1000に含まれるか
                    }
                    
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj >= 0
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj < 27750

                    behav_matrix[jjj] = np.array(list(behavior.values()))
                    
            # ============================================
            # 8. 過去15トライアルの行動データを収集
            # ============================================
            # ============================================
            # 8. 過去15トライアルの行動データを収集
            # ============================================
            past_behav_matrix = np.ones((15, 17))*-1
            jjj=-1
            for j in range(1,16):  # 1〜15トライアル前
                jj = i-j
                jjj += 1

                if jj >= 0:
                    # NaNを-1に変換（現在のトライアルと同じ処理）
                    iscorrect = behav.iloc[jj]['ISCORRECT']
                    if np.isnan(iscorrect): iscorrect = -1

                    isoldcurrent = behav.iloc[jj]['ISOLDCURRENT']
                    if np.isnan(isoldcurrent): isoldcurrent = -1

                    iscorrectcurrent = behav.iloc[jj]['ISCORRECTCURRENT']
                    if np.isnan(iscorrectcurrent): iscorrectcurrent = -1

                    rt = behav.iloc[jj]['RT']
                    if np.isnan(rt): rt = -1

                    changemind = behav.iloc[jj]['CHANGEMIND']
                    if np.isnan(changemind): changemind = -1

                    button = behav.iloc[jj]['BUTTON']
                    if np.isnan(button): button = -1

                    total1 = behav.iloc[jj]['TOTAL1']
                    if np.isnan(total1): total1 = -1

                    total2 = behav.iloc[jj]['TOTAL2']
                    if np.isnan(total2): total2 = -1
                    
                    coco73 = int(behav.iloc[jj]['73KID'])-1
                    assert coco73 >= 0 and coco73 < 730000

                    behavior = {
                        "cocoidx": coco73, #0
                        "subject": sub+1,                          #1
                        "session": int(behav.iloc[jj]['SESSION']), #2
                        "run": int(behav.iloc[jj]['RUN']),         #3
                        "trial": int(behav.iloc[jj]['TRIAL']),     #4
                        "global_trial": (int(behav.iloc[jj]['SESSION'])-1)*750 + jj,        #5
                        "time": int(behav.iloc[jj]['TIME']),       #6
                        "isold": int(behav.iloc[jj]['ISOLD']),     #7
                        "iscorrect": iscorrect,                    #8
                        "rt": rt, # 0 = no RT                      #9
                        "changemind": changemind,                  #10
                        "isoldcurrent": isoldcurrent,              #11
                        "iscorrectcurrent": iscorrectcurrent,      #12
                        "total1": total1,   #13
                        "total2": total2,   #14
                        "button": button,                          #15
                        "shared1000": shared1000[int(behav.iloc[jj]['73KID'])-1], #16
                    }
                    
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj >= 0
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj < 27750

                    past_behav_matrix[jjj] = np.array(list(behavior.values()))
                    
            # ============================================
            # 9. 未来15トライアルの行動データを収集
            # ============================================
            future_behav_matrix = np.ones((15, 17))*-1
            jjj=-1
            for j in range(1,16):  # 1〜15トライアル後
                jj = i+j
                jjj += 1

                if jj >= 0 and jj<750:  # セッション内に収まる範囲のみ
                    # NaNを-1に変換
                    iscorrect = behav.iloc[jj]['ISCORRECT']
                    if np.isnan(iscorrect): iscorrect = -1

                    isoldcurrent = behav.iloc[jj]['ISOLDCURRENT']
                    if np.isnan(isoldcurrent): isoldcurrent = -1

                    iscorrectcurrent = behav.iloc[jj]['ISCORRECTCURRENT']
                    if np.isnan(iscorrectcurrent): iscorrectcurrent = -1

                    rt = behav.iloc[jj]['RT']
                    if np.isnan(rt): rt = -1

                    changemind = behav.iloc[jj]['CHANGEMIND']
                    if np.isnan(changemind): changemind = -1

                    button = behav.iloc[jj]['BUTTON']
                    if np.isnan(button): button = -1

                    total1 = behav.iloc[jj]['TOTAL1']
                    if np.isnan(total1): total1 = -1

                    total2 = behav.iloc[jj]['TOTAL2']
                    if np.isnan(total2): total2 = -1
                    
                    coco73 = int(behav.iloc[jj]['73KID'])-1
                    assert coco73 >= 0 and coco73 < 730000

                    behavior = {
                        "cocoidx": coco73, #0
                        "subject": sub+1,                          #1
                        "session": int(behav.iloc[jj]['SESSION']), #2
                        "run": int(behav.iloc[jj]['RUN']),         #3
                        "trial": int(behav.iloc[jj]['TRIAL']),     #4
                        "global_trial": (int(behav.iloc[jj]['SESSION'])-1)*750 + jj,        #5
                        "time": int(behav.iloc[jj]['TIME']),       #6
                        "isold": int(behav.iloc[jj]['ISOLD']),     #7
                        "iscorrect": iscorrect,                    #8
                        "rt": rt, # 0 = no RT                      #9
                        "changemind": changemind,                  #10
                        "isoldcurrent": isoldcurrent,              #11
                        "iscorrectcurrent": iscorrectcurrent,      #12
                        "total1": total1,   #13
                        "total2": total2,   #14
                        "button": button,                          #15
                        "shared1000": shared1000[int(behav.iloc[jj]['73KID'])-1], #16
                    }
                    
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj >= 0
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj < 27750

                    future_behav_matrix[jjj] = np.array(list(behavior.values()))

            # ============================================
            # 10. 同じ画像を見た過去のトライアル（最大3個）の行動データを収集
            # ============================================
            olds_behav_matrix = np.ones((3, 17))*-1
            jjj=-1
            for j in range(3):
                jj = trial_numbers[j]  # 同じ画像を見たトライアル番号

                if jj>=0:  # 有効なトライアル番号の場合
                    jjj += 1
                    # トライアル番号からセッションとセッション内トライアルを計算
                    old_session = int(np.floor(jj / 750)) + 1
                    old_trial = jj % 750
                    behav = globals()[f'behav_ses{old_session}']
                    jj = old_trial

                    # NaNを-1に変換
                    iscorrect = behav.iloc[jj]['ISCORRECT']
                    if np.isnan(iscorrect): iscorrect = -1

                    isoldcurrent = behav.iloc[jj]['ISOLDCURRENT']
                    if np.isnan(isoldcurrent): isoldcurrent = -1

                    iscorrectcurrent = behav.iloc[jj]['ISCORRECTCURRENT']
                    if np.isnan(iscorrectcurrent): iscorrectcurrent = -1

                    rt = behav.iloc[jj]['RT']
                    if np.isnan(rt): rt = -1

                    changemind = behav.iloc[jj]['CHANGEMIND']
                    if np.isnan(changemind): changemind = -1

                    button = behav.iloc[jj]['BUTTON']
                    if np.isnan(button): button = -1

                    total1 = behav.iloc[jj]['TOTAL1']
                    if np.isnan(total1): total1 = -1

                    total2 = behav.iloc[jj]['TOTAL2']
                    if np.isnan(total2): total2 = -1
                    
                    coco73 = int(behav.iloc[jj]['73KID'])-1
                    assert coco73 >= 0 and coco73 < 730000

                    behavior = {
                        "cocoidx": coco73, #0
                        "subject": sub+1,                          #1
                        "session": int(behav.iloc[jj]['SESSION']), #2
                        "run": int(behav.iloc[jj]['RUN']),         #3
                        "trial": int(behav.iloc[jj]['TRIAL']),     #4
                        "global_trial": (int(behav.iloc[jj]['SESSION'])-1)*750 + jj,        #5
                        "time": int(behav.iloc[jj]['TIME']),       #6
                        "isold": int(behav.iloc[jj]['ISOLD']),     #7
                        "iscorrect": iscorrect,                    #8
                        "rt": rt, # 0 = no RT                      #9
                        "changemind": changemind,                  #10
                        "isoldcurrent": isoldcurrent,              #11
                        "iscorrectcurrent": iscorrectcurrent,      #12
                        "total1": total1,   #13
                        "total2": total2,   #14
                        "button": button,                          #15
                        "shared1000": shared1000[int(behav.iloc[jj]['73KID'])-1], #16
                    }
                    
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj >= 0
                    assert (int(behav.iloc[jj]['SESSION'])-1)*750 + jj < 27750

                    olds_behav_matrix[jjj] = np.array(list(behavior.values()))

            # ============================================
            # 11. データをWebDataset形式で保存（trainまたはtest）
            # ============================================
            behav = globals()[f'behav_ses{sess}']
            
            # Shared1000に含まれるかどうかで保存先を分ける
            if shared1000[int(behav.iloc[i]['73KID'])-1]:
                abs_shared1000_cnt += 1  # テストセットのカウンター
            else:
                abs_notshared1000_cnt += 1  # 訓練セットのカウンター
                
            with torch.no_grad():  # 勾配計算不要（メモリ節約）
                if shared1000[int(behav.iloc[i]['73KID'])-1]:
                    # テストセット（Shared1000）に保存
                    sink1.write({
                        "__key__": "sample%09d" % abs_shared1000_cnt,
                        "behav.npy": behav_matrix,           # 現在のトライアル
                        "past_behav.npy": past_behav_matrix, # 過去15トライアル
                        "future_behav.npy": future_behav_matrix, # 未来15トライアル
                        "olds_behav.npy": olds_behav_matrix,     # 同じ画像の過去トライアル
                    })
                    assert behav_matrix[-1,0] < 73000
                else:
                    # 訓練セットに保存
                    sink2.write({
                        "__key__": "sample%09d" % abs_notshared1000_cnt,
                        "behav.npy": behav_matrix,
                        "past_behav.npy": past_behav_matrix,
                        "future_behav.npy": future_behav_matrix,
                        "olds_behav.npy": olds_behav_matrix,
                    })
                    assert behav_matrix[-1,0] < 73000
        sink2.close()  # セッションごとの訓練用tarファイルを閉じる
    sink1.close()  # テスト用tarファイルを閉じる
    
    print(time.strftime("\nCurrent time: %H:%M:%S", time.localtime())) 